In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
from sqlalchemy import create_engine

In [2]:
path = r'T:\MPO\TIP\TIP FY24-27\Maps\Data'

In [3]:
file = path + '\\MTIP24_27_List.xlsx'

In [4]:
data = pd.read_excel(file)

In [5]:
data.columns

Index(['Geo', 'Project Name', 'MTIP ID #', 'Project Description', 'Work Type',
       'Perf. Meas.', 'RTP Project Number / Ref.', 'In AQ CATS?', 'In\nAQMA?',
       'Air Quality Status', 'STIP Key', 'FFY', 'Phase', 'Phase Status',
       'Federal Funding ', 'Federal Funding Source', 'Federal Req. Match',
       'Federal Req. Match Source', 'Total Fed+ Req Match', 'Other Funding',
       'Other Funding Source', 'Total All Sources'],
      dtype='object')

In [6]:
data['Air Quality Status'].unique()

array(['EXEMPT / Other-Planning and Technical Studies (IAC conf 4/26/20)',
       nan, 'EXEMPT / Other-Planning and Technical Studies',
       'N/A (IAC conf 4/26/20)', 'N/A (IAC conf 11/3/21)',
       'EXEMPT / Air Quality - Bicycle and Pedestrian facilities',
       'Outside PM10 air quality maintenance area',
       'Outside PM10 air quality maintenance area (IAC conf 11/3/21)',
       'EXEMPT / Safety - Highway Safety Improvement Program implementation',
       'EXEMPT / Other - Specific activities which do not involve or lead directly to construction',
       'EXEMPT / Other - Planning and Technical Studies',
       'EXEMPT / Safety - Projects that correct, improve, or eliminate a hazardous location or feature',
       'Part of K16223, project level conformity was approved in interagency meeting 12/23/2020',
       'EXEMPT / Safety - Highway Safety Improvement Program implementation; Air Quality - Bicycle and Pedestrian facilities (IAC conf 4/26/20)',
       'EXEMPT / Safety - Pro

In [7]:
def getAQvars(x, var=['AQ Exempt?', 'AQ Status', 'IAC']):
    if str(x) == 'nan':
        res = None
    elif var == 'AQ Exempt?':
        if 'EXEMPT' in x:
            res = 'Yes'
        else:
            res = 'No'
    
    elif var == 'IAC':
        if 'IAC' in x:
            res = 'IAC ' + x.split(' (IAC ')[1].split(')')[0]
        else:
            res = 'N/A'
    
    elif var == 'AQ Status':
        if 'EXEMPT' in x:
            if 'IAC' in x:
                res = x.split('EXEMPT / ')[1].split(' (IAC')[0]
            else:
                res = x.split('EXEMPT / ')[1]
        elif 'IAC' in x:
            res = x.split(' (IAC')[0]
        else:
            res = x
    
    return res

In [8]:
data['AQ Exempt?'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='AQ Exempt?')) 

In [9]:
data['AQ Status'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='AQ Status')) 

In [10]:
data['IAC'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='IAC')) 

In [11]:
points = gpd.read_file(os.path.join(path, 'FY24_27_points.shp'))

In [12]:
lines = gpd.read_file(os.path.join(path, 'FY24_27_lines.shp'))

In [13]:
# read data from RLIDgeo
engine = create_engine(   
"mssql+pyodbc:///?odbc_connect="
"Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B"
"Server%3Drliddb.int.lcog.org%2C5433%3B"
"Database%3DRLIDGeo%3B"
"Trusted_Connection%3Dyes%3B"
"ApplicationIntent%3DReadWrite%3B"
"WSID%3Dclwrk4087.int.lcog.org%3B")

In [14]:
rep_sql = '''
SELECT 
repdist AS id,
repname AS name,
Shape.STAsBinary() AS geometry
FROM dbo.StateRepDist;
'''

In [15]:
sen_sql = '''
SELECT 
sendist AS id,
senname AS name,
Shape.STAsBinary() AS geometry
FROM dbo.StateSenDist;
'''

In [16]:
StateRepDist = gpd.GeoDataFrame.from_postgis(rep_sql, engine, geom_col='geometry')

In [17]:
StateRepDist.crs = "EPSG:2914"

In [18]:
StateSenDist = gpd.GeoDataFrame.from_postgis(sen_sql, engine, geom_col='geometry') 

In [19]:
StateSenDist.crs = "EPSG:2914"

In [20]:
StateRepDist = StateRepDist.to_crs(epsg=2992)

In [21]:
StateSenDist = StateSenDist.to_crs(epsg=2992)

In [22]:
# points in polygons
def get_pip(points, polygon):
    id_list = list(polygon.id)
    df = pd.DataFrame().reindex_like(points).dropna()
    for ID in id_list:
        pol = (polygon.loc[polygon.id==ID])
        pol.reset_index(drop = True, inplace = True)
        pip_mask = points.within(pol.loc[0, 'geometry'])
        pip_data = points.loc[pip_mask].copy()
        pip_data['id']= ID
        df = df.append(pip_data)
    df.reset_index(inplace=True, drop=True)
    df = df.drop(columns='geometry')
    return df

In [23]:
points_in_sendist = get_pip(points, StateSenDist)

In [24]:
points_in_sendist.rename(columns={'id':'Senator District'}, inplace=True)

In [25]:
points_in_repdist = get_pip(points, StateRepDist)

In [26]:
points_in_repdist.rename(columns={'id':'Representative District'}, inplace=True)

In [27]:
lines_in_sendist = gpd.sjoin(lines, StateSenDist)

In [28]:
lines_in_sendist.rename(columns={'id':'Senator District'}, inplace=True)

In [29]:
lines_in_repdist = gpd.sjoin(lines, StateRepDist)

In [30]:
lines_in_repdist.rename(columns={'id':'Representative District'}, inplace=True)

In [31]:
len(points.STIP_Key.unique()) + len(lines.STIP_Key.unique()) 

31

In [32]:
np.concatenate((points.STIP_Key.unique(), lines.STIP_Key.unique())).shape[0]

31

In [34]:
points_in_poly = points_in_repdist.drop_duplicates(ignore_index=True)[['STIP_Key', 'Representative District']].merge(points_in_sendist.drop_duplicates(ignore_index=True)[['STIP_Key', 'Senator District']], 
                                                                            on ='STIP_Key')

In [35]:
lines_in_poly = lines_in_repdist.drop_duplicates(ignore_index=True)[['STIP_Key', 'Representative District']].merge(lines_in_sendist.drop_duplicates(ignore_index=True)[['STIP_Key', 'Senator District']], 
                                                                            on ='STIP_Key')

In [36]:
df = points_in_poly.append(lines_in_poly, ignore_index=True)

In [37]:
df.rename(columns={'STIP_Key':'STIP Key'}, inplace=True)

In [43]:
df['STIP Key'] = df['STIP Key'].astype(np.int64)

In [44]:
ndata = data.merge(df, on='STIP Key')

In [49]:
ndata.to_excel(os.path.join(path, 'MTIP24_27_List_Updated.xlsx'), 
               sheet_name='2024-2027 Project List', 
               index=False,
               engine='io.excel.xlsx.writer')  

ValueError: No Excel writer 'io.excel.xlsx.writer'